# 1. Data Filtering

* Objetivo: Crear un "Ground Truth" de texto humano puro.

* Estrategia: Leer un dataset masivo de noticias y extraer una muestra aleatoria de artículos publicados antes de 2020 (antes del auge de GPT-3).

## 1.1. Configuración inicial

En esta sección importamos las librerías necesarias para la manipulación de datos (`pandas`) y gestión del sistema (`os`). 

Definimos constantes críticas:
* `INPUT_FILE`: Ruta al dataset masivo descargado de Kaggle.
* `CHUNK_SIZE`: Tamaño del bloque de lectura (50,000 filas) para evitar desbordar la memoria RAM.
* `DATE_CUTOFF`: La fecha límite (**01-01-2020**) que actúa como frontera para considerar un texto como "100% Humano".

In [ ]:
# Importamos librerías necesarias
import pandas as pd
import os
import random

# Configuración de rutas y parámetros
INPUT_FILE = '../data/1_raw/all-the-news-2-1.csv' 

# Ruta donde guardaremos la muestra pequeña
OUTPUT_FILE =  '../data/1_raw/all-the-news-sample.csv' 

# Parámetros de muestreo
SAMPLE_SIZE = 5000       
CHUNK_SIZE = 50000       
DATE_CUTOFF = '2020-01-01' 

## 1.2. Función principal de filtrado y muestreo

Aquí definimos la lógica central del script (`create_sample`). Dado que el archivo original pesa ~8GB, no podemos cargarlo entero.

**El algoritmo funciona así:**
1.  **Iteración:** Recorre el archivo CSV original bloque a bloque (*chunk by chunk*).
2.  **Filtrado:** En cada bloque, descarta las noticias posteriores a 2020 y aquellas sin contenido válido.
3.  **Acumulación:** Guarda temporalmente los candidatos válidos.
4.  **Muestreo:** Una vez recorrido todo el archivo, selecciona aleatoriamente `SAMPLE_SIZE` (5,000) artículos para garantizar representatividad estadística.

In [ ]:
def create_sample():
    """
    Lee el archivo masivo por bloques, filtra noticias antiguas y selecciona
    una muestra aleatoria para el estudio.
    """
    
    # Lista temporal para guardar candidatos válidos
    valid_articles_pool = []
    
    # Procesamos el archivo por trozos (chunks)
    try:
        chunks = pd.read_csv(INPUT_FILE, chunksize=CHUNK_SIZE, parse_dates=['date'], low_memory=False)
        
        total_processed = 0
        
        for i, chunk in enumerate(chunks):
            # 1. Limpieza básica de fechas nulas
            chunk = chunk.dropna(subset=['date', 'article'])
            
            # 2. Convertir fecha a datetime (por seguridad, si parse_dates falla)
            chunk['date'] = pd.to_datetime(chunk['date'], errors='coerce')
            
            # Nos aseguramos de que sea anterior a 2020 Y que no sea muy antigua (ej. 2015 para modernidad)
            mask = (chunk['date'] < DATE_CUTOFF) & (chunk['date'] >= '2015-01-01')
            valid_chunk = chunk[mask]
            
            # 4. Optimización: Guardamos solo columnas necesarias para ahorrar memoria
            cols_to_keep = ['date', 'title', 'article', 'publication', 'author']
            
            # Aseguramos que existan las columnas antes de filtrar
            existing_cols = [c for c in cols_to_keep if c in valid_chunk.columns]
            
            valid_articles_pool.append(valid_chunk[existing_cols])
            
            total_processed += len(chunk)
            print(f"   ⏳ Procesadas {total_processed:,} filas... (Encontrados {len(valid_chunk)} válidos en este bloque)")


    except Exception as e:
        print(f"Ocurrió un error leyendo el archivo: {e}")
        return

    # Concatenar todos los trozos válidos en un solo DataFrame temporal
    full_valid_df = pd.concat(valid_articles_pool, ignore_index=True)
    
    print(f"Total de artículos válidos (Pre-2020) encontrados: {len(full_valid_df):,}")

    # 5. Muestreo aleatorio
    if len(full_valid_df) > SAMPLE_SIZE:
        print(f"🎲 Seleccionando aleatoriamente {SAMPLE_SIZE} noticias...")
        sampled_df = full_valid_df.sample(n=SAMPLE_SIZE, random_state=42) # random_state para reproducibilidad
    else:
        print(f"⚠️ Advertencia: Hay menos de {SAMPLE_SIZE} artículos. Usando todos los disponibles.")
        sampled_df = full_valid_df

    # 6. Guardardamos el DataFrame muestreado a CSV
    print(f"Guardando CSV en {OUTPUT_FILE}...")
    sampled_df.to_csv(OUTPUT_FILE, index=False)

## 1.3. Ejecución del Script

Bloque de entrada estándar. Ejecuta la función `create_sample()` definida anteriormente. 
Si el archivo de salida ya existe, este proceso lo sobrescribirá con una nueva muestra aleatoria.

In [ ]:
if __name__ == "__main__":
    create_sample()